In [2]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 24 × AMD Ryzen 9 3900X 12-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver2)
  Threads: 18 on 24 virtual cores
Environment:
  JULIA_PKG_SERVER = pkg.julialang.org


In [3]:
]precompile

Precompiling project...
  ✓ Mods
  ✓ OpenMPI_jll
  ✓ Mocking
  ✓ SimplePartitions
  ✓ FilePaths
  ✓ Peaks
  ✓ NLopt
  ✓ MakieCore
  ✓ StableHashTraits
  ✓ AxisArrays
  ✓ OnlineStatsBase
  ✓ Polynomials → PolynomialsChainRulesCoreExt
  ✓ FiniteDiff
  ✓ IntervalArithmetic
  ✓ FastGaussQuadrature
  ✓ StatsModels
  ✓ SimpleRandom
  ✓ DataFramesMeta
  ✓ AccurateArithmetic
  ✓ HDF5_jll
  ✓ Clustering
  ✓ StrideArraysCore
  ✓ KernelDensity
  ✓ IntervalArithmetic → IntervalArithmeticRecipesBaseExt
  ✓ DatasetManager
  ✓ FileIO
  ✓ RegressionFormulae
  ✓ Polynomials → PolynomialsMakieCoreExt
  ✓ FiniteDiff → FiniteDiffBandedMatricesExt
  ✓ StandardizedPredictors
  ✓ IntervalArithmetic → IntervalArithmeticDiffRulesExt
  ✓ OnlineStats
  ✓ GLM
  ✓ Polyester
  ✓ ImageAxes
  ✓ QOI
  ✓ OpenEXR
  ✓ Effects
  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  ✓ SimplePolynomials
  ✓ Sixel
  ✓ JpegTurbo
  ✓ FastBroadcast
  ✓ ImageMetadata
  ✓ GeometryBasics
  ✓ LinearAlgebraX
  ✓ NLSolversBase
  ✓ Effects → Eff

In [11]:
using DatasetManager, LabDataSources, C3D, CSV, DataFrames, DataFramesMeta
using C3D: deleteanalog!

In [12]:
if Sys.iswindows()
    disk = "G:/"
else
    if isdir("/mnt/data/")
        disk = "/mnt/data/"
    else
        disk = "/media/allen/project-data"
    end
end

datapath = joinpath(disk, "projects/hirams-study/data")
c3dpath = joinpath(datapath, "raw/vicon")

subsets = [
    DataSubset("main", Source{C3DFile}, c3dpath, "PD [0-9]*/Patient [0-9]*/Session [0-2]/*.c3d")
    DataSubset("main", Source{C3DFile}, c3dpath, "Control [0-9]*/Patient [0-9]*/Session [0-2]/*.c3d")
]

labels = Dict(
    :group => ["PD", "Control"],
    :groupid => r"(?<=PD |Control )\d+",
    :subject => r"(?<=Patient )\d+",
    :session => r"(?<=Session )\d+",
    :task => r"(?<!cal[-_])(rrt|sip)"i => lowercase,
    :posture => r"(sit|stand)"i => lowercase,
    :side => r"(right|left|both)"i => lowercase,
    :cued => r"(?<=[-_])N?C"i => lowercase
)

conds = TrialConditions((:group,:groupid,:subject,:session,:task,:posture,:side,:cued), labels; types=Dict(:groupid => Int, :subject => Int, :session => Int))

trials = findtrials(subsets, conds)

labels = Dict(
    :group => ["PD", "Control"],
    :groupid => r"(?<=PD |Control )\d+",
    :subject => r"(?<=Patient )\d+",
    :session => r"(?<=Session )\d+",
    :freq_cal => r"(?<=cal[-_])(sip|rrt)(?=[-_]freq)"i => ((s) -> "cal_$(lowercase(s))_freq") => ["cal_sip_freq", "cal_rrt_freq"]
)

conds = TrialConditions((:group,:groupid,:subject,:session,:freq_cal), labels; required=(:group,:groupid,:subject,:session,), types=Dict(:groupid => Int, :subject => Int, :session => Int))

subsets = [
    DataSubset("freq_cal", Source{C3DFile}, c3dpath, "PD [0-9]*/Patient [0-9]*/Session [0-2]/*.c3d"; dependent=true)
    DataSubset("freq_cal", Source{C3DFile}, c3dpath, "Control [0-9]*/Patient [0-9]*/Session [0-2]/*.c3d"; dependent=true)
]

trials = findtrials!(trials, subsets, conds)

demograph = CSV.read("../data/demographics.csv", DataFrame)
DatasetManager.addcondition!.(trials, :bodyweight => (t -> @subset(demograph, $"Subject Code" .== subject(t))[1, "Weight (kg)"]))

recodecondition!.(trials, :cued => c -> c == "c" ? true : false)
delete!.(conditions.(trials), :posture)
delete!.(conditions.(trials), :side)

summarize(trials; verbosity=25, ignoreconditions=(:groupid,:group,:bodyweight))

┌ Error: DuplicateSourceError: Found "freq_cal" source file …/PD 02/Patient 4/Session 1/CAL_RRT_FREQ.c3d for Trial(4, "RRT_STAND_BOTH_C", 7 conditions, 2 sources) which already has a "freq_cal" source at "…/PD 02/Patient 4/Session 1/CAL_RRT_FREQ-2.c3d"
└ @ DatasetManager ~/.julia/dev/DatasetManager/src/trial.jl:735
┌ Error: DuplicateSourceError: Found "freq_cal" source file …/PD 02/Patient 4/Session 1/cal-rrt-freq-1.c3d for Trial(4, "RRT_STAND_BOTH_C", 7 conditions, 3 sources) which already has a "freq_cal" source at "…/PD 02/Patient 4/Session 1/CAL_RRT_FREQ-2.c3d"
└ @ DatasetManager ~/.julia/dev/DatasetManager/src/trial.jl:735
┌ Error: DuplicateSourceError: Found "freq_cal" source file …/PD 20/Patient 27/Session 1/cal-rrt-freq-4.c3d for Trial(27, "rrt-stand-both-c", 7 conditions, 2 sources) which already has a "freq_cal" source at "…/PD 20/Patient 27/Session 1/cal-rrt-freq-3.c3d"
└ @ DatasetManager ~/.julia/dev/DatasetManager/src/trial.jl:735


Subjects:
 └ 40: 1  2  3  4  5  6  7  8  9  10  11  …  33  34  35  36  37  38  39  40
Trials:
 ├ 283 trials
 └ Trials per subject:
   ├ 8: 32 subjects (80%)
   ├ 4: 6 subjects (15%)
   ├ 2: 1 subject (2%)
   └ 1: 1 subject (2%)
Conditions:
 ├ Observed levels:
 │ ├ task => ["rrt", "sip"]
 │ ├ cued => Bool[1, 0]
 │ └ session => [1, 2]
 └ Unique level combinations observed: 8 (full factorial)
     task │  cued │ session │ # trials 
    ──────┼───────┼─────────┼──────────
      rrt │  true │       1 │ 39       
      rrt │ false │       1 │ 38       
      sip │  true │       1 │ 37       
      sip │ false │       1 │ 37       
      rrt │  true │       2 │ 33       
      rrt │ false │       2 │ 33       
      sip │  true │       2 │ 33       
      sip │ false │       2 │ 33       
Sources:
 ├ "cal_rrt_freq" => Source{C3DFile}, 283 trials (100%)
 ├ "cal_sip_freq" => Source{C3DFile}, 283 trials (100%)
 └ "main" => Source{C3DFile}, 283 trials (100%)


In [16]:
emgsigs = ["Voltage.LAD", "Voltage.LBF", "Voltage.LBIC", "Voltage.LGM", "Voltage.LRF",
    "Voltage.LTA", "Voltage.LTRIC", "Voltage.LUT", "Voltage.RAD", "Voltage.RBF",
    "Voltage.RBIC", "Voltage.RGM", "Voltage.RRF", "Voltage.RTA", "Voltage.RTRIC",
    "Voltage.RUT"];

In [19]:
foreach(trials) do t
    requiresource!(t, "main" => Source{C3DFile}(); force=true, deps=(Source{C3DFile},)) do trial, src, deps
        p, io = mktemp()
        f = readsource(trial, "main")
        deleteanalog!.(Ref(f), emgsigs)
        writec3d(io, f)
        close(io)
        t.sources["main"] = Source{C3DFile}(p)
    end
    requiresource!(t, "cal_rrt_freq" => Source{C3DFile}(); force=true, deps=(Source{C3DFile},)) do trial, src, deps
        p, io = mktemp()
        f = readsource(trial, "cal_rrt_freq")
        deleteanalog!.(Ref(f), emgsigs)
        writec3d(io, f)
        close(io)
        t.sources["cal_rrt_freq"] = Source{C3DFile}(p)
    end
    requiresource!(t, "cal_sip_freq" => Source{C3DFile}(); force=true, deps=(Source{C3DFile},)) do trial, src, deps
        p, io = mktemp()
        f = readsource(trial, "cal_sip_freq")
        deleteanalog!.(Ref(f), emgsigs)
        writec3d(io, f)
        close(io)
        t.sources["cal_sip_freq"] = Source{C3DFile}(p)
    end
end

In [29]:
export_trials(trials, "../data", ["main"]) do trial, srcname
    name = "$(conditions(trial)[:task])-"*(conditions(trial)[:cued] ? "cued" : "uncued")
    return "$(conditions(trial)[:group]) $(conditions(trial)[:groupid])/Patient $(subject(trial))/Session $(conditions(trial)[:session])/$name.c3d"
end